<a href="https://colab.research.google.com/github/joaohl19/premierleague_2018-19/blob/main/preparo_premierleague_2018_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preparo de dados da edição 2018/19 da Premier League para análise**


O preparo tem como intuito enriquecer o dataset Partidas com dados importantes através da inferência de dados a partir dos datasets disponíveis e do uso de webscraping. O intuito final é usar esses dados no treinamento de modelos preditivos.

Esses dados úteis podem variar de acordo com o que o modelo está tentando prever(gols, ganhador da partida, cartões...)

EX : Saber a média de gols de um time no campeonato é bastante útil no momento de prever quantos gols ele irá marcar em determinado jogo.


Os datasets usados foram disponibilizados pelo site https://footystats.org e podem ser baixados diretamente pelos links abaixo

Dataset Partidas : https://footystats.org/c-dl.php?type=matches&comp=1625

Dataset Times 1 : https://footystats.org/c-dl.php?type=teams&comp=1625

Dataset Times 2 : https://footystats.org/c-dl.php?type=teams2&comp=1625

## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import requests as r
from bs4 import BeautifulSoup as bs

import os
import warnings
import copy

## Datasets

### Dataset Partidas


In [2]:
partidas = pd.read_csv('/content/england-premier-league-matches-2018-to-2019-stats.csv')
partidas

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1533927600,Aug 10 2018 - 7:00pm,complete,74439,Manchester United,Leicester City,Andre Marriner,1,0.00,0.00,...,1.37,4.98,9.81,1.33,2.00,3.60,7.50,2.05,1.69,Old Trafford (Manchester)
1,1533987000,Aug 11 2018 - 11:30am,complete,51749,Newcastle United,Tottenham Hotspur,Martin Atkinson,1,0.00,0.00,...,4.51,3.77,1.84,1.31,1.95,3.50,5.75,1.83,1.87,St. James' Park (Newcastle upon Tyne)
2,1533996000,Aug 11 2018 - 2:00pm,complete,10353,AFC Bournemouth,Cardiff City,Kevin Friend,1,0.00,0.00,...,2.03,3.51,3.96,1.31,1.95,3.45,6.70,1.83,1.83,Vitality Stadium (Bournemouth- Dorset)
3,1533996000,Aug 11 2018 - 2:00pm,complete,24821,Fulham,Crystal Palace,Mike Dean,1,0.00,0.00,...,2.31,3.44,3.26,1.28,1.87,3.20,7.00,1.71,2.00,Craven Cottage (London)
4,1533996000,Aug 11 2018 - 2:00pm,complete,24121,Huddersfield Town,Chelsea,Chris Kavanagh,1,0.00,0.00,...,7.47,4.27,1.51,1.29,1.91,3.30,6.95,2.00,1.71,John Smith's Stadium (Huddersfield- West Yorks...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1557669600,May 12 2019 - 2:00pm,complete,53331,Liverpool,Wolverhampton Wanderers,Martin Atkinson,38,2.78,1.28,...,1.32,5.80,9.00,1.18,1.57,2.45,4.75,1.95,1.83,Anfield (Liverpool)
376,1557669600,May 12 2019 - 2:00pm,complete,74457,Manchester United,Cardiff City,Jonathan Moss,38,2.00,0.61,...,1.28,6.20,10.00,1.12,1.41,2.00,3.60,1.74,2.05,Old Trafford (Manchester)
377,1557669600,May 12 2019 - 2:00pm,complete,30367,Southampton,Huddersfield Town,Lee Probert,38,1.22,0.33,...,1.41,4.90,7.50,1.19,1.61,2.50,4.75,1.80,2.00,St. Mary's Stadium (Southampton- Hampshire)
378,1557669600,May 12 2019 - 2:00pm,complete,60124,Tottenham Hotspur,Everton,Andre Marriner,38,2.06,1.06,...,2.10,3.50,3.55,1.22,1.71,2.80,6.00,1.59,2.30,Tottenham Hotspur Stadium (London)


In [3]:
colunas = partidas.columns
colunas

Index(['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)',
       'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_goal_timings', 'away_team_goal_timings',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'away_team_second_half_cards',
       'home_team_shots', 'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_te

In [4]:
# Os únicos valores NaN ocorreram nas colunas que indicam os momentos em que a
# equipe marcou gol
for coluna in colunas:
  NaN = partidas[coluna].isna().sum()
  if(NaN > 0):
    print(coluna , NaN, sep = ": ")

home_team_goal_timings: 88
away_team_goal_timings: 119


In [5]:
partidas.fillna(value = "No goal", inplace=True)

### Dataset Times 1


In [6]:
times1 = pd.read_csv('/content/england-premier-league-teams-2018-to-2019-stats.csv')
times1

,team_name,common_name,season,country,matches_played,matches_played_home,matches_played_away,suspended_matches,wins,wins_home,...,goals_conceded_min_61_to_70,goals_conceded_min_71_to_80,goals_conceded_min_81_to_90,draw_percentage_overall,draw_percentage_home,draw_percentage_away,loss_percentage_ovearll,loss_percentage_home,loss_percentage_away,over145_corners_percentage
0,Arsenal FC,Arsenal,2018/2019,England,38,19,19,0,21,14,...,12,1,8,18,16,21,26,11,42,5
1,Tottenham Hotspur FC,Tottenham Hotspur,2018/2019,England,38,19,19,0,23,12,...,4,8,10,5,11,0,34,26,42,11
2,Manchester City FC,Manchester City,2018/2019,England,38,19,19,0,32,18,...,4,2,3,5,0,11,11,5,16,5
3,Leicester City FC,Leicester City,2018/2019,England,38,19,19,0,15,8,...,7,1,9,18,16,21,42,42,42,5
4,Crystal Palace FC,Crystal Palace,2018/2019,England,38,19,19,0,14,5,...,8,8,11,18,26,11,45,47,42,11
5,Everton FC,Everton,2018/2019,England,38,19,19,0,15,10,...,10,5,4,24,21,26,37,26,47,11
6,Burnley FC,Burnley,2018/2019,England,38,19,19,0,11,7,...,10,4,15,18,11,26,53,53,53,0
7,Southampton FC,Southampton,2018/2019,England,38,19,19,0,9,5,...,5,4,14,32,42,21,45,32,58,8
8,AFC Bournemouth,AFC Bournemouth,2018/2019,England,38,19,19,0,13,8,...,8,6,11,16,26,5,50,32,68,11
9,Manchester United FC,Manchester United,2018/2019,England,38,19,19,0,19,10,...,7,7,7,24,32,16,26,16,37,8


In [7]:
times1.columns[0 : 100]

Index(['team_name', 'common_name', 'season', 'country', 'matches_played',
       'matches_played_home', 'matches_played_away', 'suspended_matches',
       'wins', 'wins_home', 'wins_away', 'draws', 'draws_home', 'draws_away',
       'losses', 'losses_home', 'losses_away', 'points_per_game',
       'points_per_game_home', 'points_per_game_away', 'league_position',
       'league_position_home', 'league_position_away', 'performance_rank',
       'goals_scored', 'goals_conceded', 'goal_difference', 'total_goal_count',
       'total_goal_count_home', 'total_goal_count_away', 'goals_scored_home',
       'goals_scored_away', 'goals_conceded_home', 'goals_conceded_away',
       'goal_difference_home', 'goal_difference_away',
       'minutes_per_goal_scored', 'minutes_per_goal_scored_home',
       'minutes_per_goal_scored_away', 'minutes_per_goal_conceded',
       'minutes_per_goal_conceded_home', 'minutes_per_goal_conceded_away',
       'clean_sheets', 'clean_sheets_home', 'clean_sheets_away'

In [8]:
times1.columns[100 : 200]

Index(['goals_scored_per_match_home', 'goals_scored_per_match_away',
       'goals_conceded_per_match', 'goals_conceded_per_match_home',
       'goals_conceded_per_match_away', 'total_goals_per_match_half_time',
       'total_goals_per_match_half_time_home',
       'total_goals_per_match_half_time_away',
       'goals_scored_per_match_half_time',
       'goals_scored_per_match_half_time_home',
       'goals_scored_per_match_half_time_away',
       'goals_conceded_per_match_half_time',
       'goals_conceded_per_match_half_time_home',
       'goals_conceded_per_match_half_time_away', 'over05_count',
       'over15_count', 'over25_count', 'over35_count', 'over45_count',
       'over55_count', 'over05_count_home', 'over15_count_home',
       'over25_count_home', 'over35_count_home', 'over45_count_home',
       'over55_count_home', 'over05_count_away', 'over15_count_away',
       'over25_count_away', 'over35_count_away', 'over45_count_away',
       'over55_count_away', 'under05_count', 'un

In [9]:
times1.columns[200: 293]

Index(['over25_half_time_percentage_home', 'over05_half_time_percentage_away',
       'over15_half_time_percentage_away', 'over25_half_time_percentage_away',
       'win_percentage', 'win_percentage_home', 'win_percentage_away',
       'home_advantage_percentage', 'clean_sheet_percentage',
       'clean_sheet_percentage_home', 'clean_sheet_percentage_away',
       'btts_percentage', 'btts_percentage_home', 'btts_percentage_away',
       'fts_percentage', 'fts_percentage_home', 'fts_percentage_away',
       'first_team_to_score_percentage', 'first_team_to_score_percentage_home',
       'first_team_to_score_percentage_away', 'clean_sheet_half_time',
       'clean_sheet_half_time_home', 'clean_sheet_half_time_away',
       'clean_sheet_half_time_percentage',
       'clean_sheet_half_time_percentage_home',
       'clean_sheet_half_time_percentage_away', 'fts_half_time',
       'fts_half_time_home', 'fts_half_time_away', 'fts_half_time_percentage',
       'fts_half_time_percentage_home', 'f

In [10]:
# Não há nenhum NaN neste dataset
for coluna in times1.columns:
  NaN = times1[coluna].isna().sum()
  if(NaN > 0):
    print(coluna , NaN, sep = ": ")

### Dataset Times 2

In [11]:
times2 = pd.read_csv('/content/england-premier-league-teams2-2018-to-2019-stats.csv')
times2

,team_name,common_name,season,country,average_attendance_overall,average_attendance_home,average_attendance_away,minutes_per_goal_scored_overall,minutes_per_goal_scored_home,minutes_per_goal_scored_away,...,btts_2h_away,btts_2h_percentage_overall,btts_2h_percentage_home,btts_2h_percentage_away,clean_sheets_2h_overall,clean_sheets_2h_home,clean_sheets_2h_away,failed_to_score_2h_overall,failed_to_score_2h_home,failed_to_score_2h_away
0,Arsenal FC,Arsenal,2018/2019,England,49412,59899,38925,47,41,55,...,6,28,26,32,18,12,6,14,4,10
1,Tottenham Hotspur FC,Tottenham Hotspur,2018/2019,England,45931,54216,37645,51,50,52,...,6,32,32,32,20,11,9,14,5,9
2,Manchester City FC,Manchester City,2018/2019,England,45914,54130,37698,36,30,45,...,3,21,26,16,27,14,13,9,2,7
3,Leicester City FC,Leicester City,2018/2019,England,34948,31851,38046,67,71,63,...,8,28,16,42,22,13,9,13,9,4
4,Crystal Palace FC,Crystal Palace,2018/2019,England,32193,25455,38932,67,90,54,...,10,39,26,53,14,8,6,15,10,5
5,Everton FC,Everton,2018/2019,England,38883,39043,38722,63,57,71,...,6,21,11,32,19,12,7,20,10,10
6,Burnley FC,Burnley,2018/2019,England,29307,20534,38080,76,71,81,...,6,28,26,32,16,8,8,22,11,11
7,Southampton FC,Southampton,2018/2019,England,33710,30083,37338,76,63,95,...,6,34,37,32,15,8,7,19,9,10
8,AFC Bournemouth,AFC Bournemouth,2018/2019,England,24718,10532,38904,61,57,66,...,2,16,21,11,14,8,6,22,10,12
9,Manchester United FC,Manchester United,2018/2019,England,56241,74498,37983,53,52,54,...,4,34,47,21,14,5,9,18,8,10


In [12]:
times2.columns[0: 100]

Index(['team_name', 'common_name', 'season', 'country',
       'average_attendance_overall', 'average_attendance_home',
       'average_attendance_away', 'minutes_per_goal_scored_overall',
       'minutes_per_goal_scored_home', 'minutes_per_goal_scored_away',
       'minutes_per_goal_conceded_overall', 'minutes_per_goal_conceded_home',
       'minutes_per_goal_conceded_away', 'highest_goals_scored_home',
       'highest_goals_scored_away', 'highest_goals_conceded_home',
       'highest_goals_conceded_away', 'half_time_points_overall',
       'half_time_points_home', 'half_time_points_away',
       'corners_recorded_matches_num_overall',
       'corners_recorded_matches_num_home', 'cornersRecorded_matches_away',
       'corners_total_per_match_overall', 'corners_total_per_match_home',
       'corners_total_per_match_away', 'corners_against_overall',
       'corners_against_home', 'corners_against_away',
       'corners_against_per_match_overall', 'corners_against_per_match_home',
      

In [13]:
times2.columns[100:200]

Index(['over45_corners_for_percentage_home',
       'over55_corners_for_percentage_home',
       'over65_corners_for_percentage_home',
       'over75_corners_for_percentage_home',
       'over85_corners_for_percentage_home', 'over25_corners_for_away',
       'over35_corners_for_away', 'over45_corners_for_away',
       'over55_corners_for_away', 'over65_corners_for_away',
       'over75_corners_for_away', 'over85_corners_for_away',
       'over25_corners_for_percentage_away',
       'over35_corners_for_percentage_away',
       'over45_corners_for_percentage_away',
       'over55_corners_for_percentage_away',
       'over65_corners_for_percentage_away',
       'over75_corners_for_percentage_away',
       'over85_corners_for_percentage_away', 'over25_corners_against_overall',
       'over35_corners_against_overall', 'over45_corners_against_overall',
       'over55_corners_against_overall', 'over65_corners_against_overall',
       'over75_corners_against_overall', 'over85_corners_against_o

In [14]:
times2.columns[200: 300]

Index(['over05_cards_total_home', 'over15_cards_total_home',
       'over25_cards_total_home', 'over35_cards_total_home',
       'over45_cards_total_home', 'over55_cards_total_home',
       'over65_cards_total_home', 'over75_cards_total_home',
       'over85_cards_total_home', 'over05_cards_total_percentage_home',
       'over15_cards_total_percentage_home',
       'over25_cards_total_percentage_home',
       'over35_cards_total_percentage_home',
       'over45_cards_total_percentage_home',
       'over55_cards_total_percentage_home',
       'over65_cards_total_percentage_home',
       'over75_cards_total_percentage_home',
       'over85_cards_total_percentage_home', 'over05_cards_total_away',
       'over15_cards_total_away', 'over25_cards_total_away',
       'over35_cards_total_away', 'over45_cards_total_away',
       'over55_cards_total_away', 'over65_cards_total_away',
       'over75_cards_total_away', 'over85_cards_total_away',
       'over05_cards_total_percentage_away',
       '

In [15]:
times2.columns[300: 400]

Index(['over15_cards_against_percentage_home',
       'over25_cards_against_percentage_home',
       'over35_cards_against_percentage_home',
       'over45_cards_against_percentage_home',
       'over55_cards_against_percentage_home',
       'over65_cards_against_percentage_home', 'over05_cards_against_away',
       'over15_cards_against_away', 'over25_cards_against_away',
       'over35_cards_against_away', 'over45_cards_against_away',
       'over55_cards_against_away', 'over65_cards_against_away',
       'over05_cards_against_percentage_away',
       'over15_cards_against_percentage_away',
       'over25_cards_against_percentage_away',
       'over35_cards_against_percentage_away',
       'over45_cards_against_percentage_away',
       'over55_cards_against_percentage_away',
       'over65_cards_against_percentage_away',
       'firstGoalScoredPercentage_overall', 'firstGoalScoredPercentage_home',
       'firstGoalScoredPercentage_away', 'shots_per_match_overall',
       'shots_per_m

In [16]:
# Não há nenhum NaN neste dataset
for coluna in times2.columns:
  NaN = times2[coluna].isna().sum()
  if(NaN > 0):
    print(coluna , NaN, sep = ": ")

## Extração e Transformação


Escolhi as coluna mais relevantes entre as colunas dos datasets Times 1 e Times 2, de forma arbitrária, imaginando que os dados seriam usados para prever se uma partida terá 2 ou mais gols



In [17]:
# Selecionando as colunas relevantes do dataset Partidas
df = partidas.loc[:, ['timestamp', 'home_team_name', 'away_team_name', 'referee', 'Home Team Pre-Match xG',
                              'Away Team Pre-Match xG', 'over_15_percentage_pre_match', 'odds_ft_over15', 'average_goals_per_match_pre_match' ]]
df['goals'] = partidas['home_team_goal_count'] + partidas['away_team_goal_count']

### Coletando as métricas importantes inferidas através do dataset Partidas

In [18]:
df_times = times1.loc[:, ['common_name','total_goal_count',
       'total_goal_count_home', 'total_goal_count_away', 'goals_scored_per_match', 'goals_scored_per_match_home', 'goals_scored_per_match_away',
       'goals_conceded_per_match', 'goals_conceded_per_match_home','goals_conceded_per_match_away','over15_count','over15_count_home', 'over15_count_away',
        'under15_count','under15_count_home', 'under15_count_away', 'clean_sheet_percentage',
       'clean_sheet_percentage_home', 'clean_sheet_percentage_away']]
df_times

,common_name,total_goal_count,total_goal_count_home,total_goal_count_away,goals_scored_per_match,goals_scored_per_match_home,goals_scored_per_match_away,goals_conceded_per_match,goals_conceded_per_match_home,goals_conceded_per_match_away,over15_count,over15_count_home,over15_count_away,under15_count,under15_count_home,under15_count_away,clean_sheet_percentage,clean_sheet_percentage_home,clean_sheet_percentage_away
0,Arsenal,124,58,66,1.92,2.21,1.63,1.34,0.84,1.84,34,18,16,4,1,3,21,37,5
1,Tottenham Hotspur,106,50,56,1.76,1.79,1.74,1.03,0.84,1.21,27,12,15,11,7,4,34,37,32
2,Manchester City,118,69,49,2.50,3.00,2.00,0.61,0.63,0.58,31,16,15,7,3,4,53,47,58
3,Leicester City,99,44,55,1.34,1.26,1.42,1.26,1.05,1.47,28,14,14,10,5,5,26,32,21
4,Crystal Palace,104,42,62,1.34,1.00,1.68,1.39,1.21,1.58,28,12,16,10,7,3,32,37,26
5,Everton,100,51,49,1.42,1.58,1.26,1.21,1.11,1.32,28,15,13,10,4,6,37,47,26
6,Burnley,113,56,57,1.18,1.26,1.11,1.79,1.68,1.89,31,17,14,7,2,5,21,26,16
7,Southampton,110,57,53,1.18,1.42,0.95,1.71,1.58,1.84,32,17,15,6,2,4,18,16,21
8,AFC Bournemouth,126,55,71,1.47,1.58,1.37,1.84,1.32,2.37,34,15,19,4,4,0,26,32,21
9,Manchester United,119,58,61,1.71,1.74,1.68,1.42,1.32,1.53,34,17,17,4,2,2,18,11,26


In [19]:
df_times.columns

Index(['common_name', 'total_goal_count', 'total_goal_count_home',
       'total_goal_count_away', 'goals_scored_per_match',
       'goals_scored_per_match_home', 'goals_scored_per_match_away',
       'goals_conceded_per_match', 'goals_conceded_per_match_home',
       'goals_conceded_per_match_away', 'over15_count', 'over15_count_home',
       'over15_count_away', 'under15_count', 'under15_count_home',
       'under15_count_away', 'clean_sheet_percentage',
       'clean_sheet_percentage_home', 'clean_sheet_percentage_away'],
      dtype='object')

In [20]:
"""
Dicionário para manter o registro das métricas importantes
de cada time em casa e fora
"""
dic_mandante = {}
dic_visitante = {}
for x in df_times['common_name']:
  dic_mandante[x] = [0,0, 0, 0]
  dic_visitante[x] = [0,0, 0, 0]
dic_mandante

{'Arsenal': [0, 0, 0, 0],
 'Tottenham Hotspur': [0, 0, 0, 0],
 'Manchester City': [0, 0, 0, 0],
 'Leicester City': [0, 0, 0, 0],
 'Crystal Palace': [0, 0, 0, 0],
 'Everton': [0, 0, 0, 0],
 'Burnley': [0, 0, 0, 0],
 'Southampton': [0, 0, 0, 0],
 'AFC Bournemouth': [0, 0, 0, 0],
 'Manchester United': [0, 0, 0, 0],
 'Liverpool': [0, 0, 0, 0],
 'Chelsea': [0, 0, 0, 0],
 'West Ham United': [0, 0, 0, 0],
 'Watford': [0, 0, 0, 0],
 'Newcastle United': [0, 0, 0, 0],
 'Cardiff City': [0, 0, 0, 0],
 'Fulham': [0, 0, 0, 0],
 'Brighton & Hove Albion': [0, 0, 0, 0],
 'Huddersfield Town': [0, 0, 0, 0],
 'Wolverhampton Wanderers': [0, 0, 0, 0]}

In [21]:
"""
Dicionário para registrar o número de partidas
jogadas por cada time
"""
lista_times = df_times['common_name']
jogos = {}
for x in lista_times:
  jogos[x] = 0
jogos

{'Arsenal': 0,
 'Tottenham Hotspur': 0,
 'Manchester City': 0,
 'Leicester City': 0,
 'Crystal Palace': 0,
 'Everton': 0,
 'Burnley': 0,
 'Southampton': 0,
 'AFC Bournemouth': 0,
 'Manchester United': 0,
 'Liverpool': 0,
 'Chelsea': 0,
 'West Ham United': 0,
 'Watford': 0,
 'Newcastle United': 0,
 'Cardiff City': 0,
 'Fulham': 0,
 'Brighton & Hove Albion': 0,
 'Huddersfield Town': 0,
 'Wolverhampton Wanderers': 0}

In [22]:
df_times = df_times.drop(columns=['common_name'])

In [23]:
# Inicializando o dicionário que vai armazenar os valores importantes
data = {}
for index, row in partidas.iterrows():
  if(row['home_team_name'] not in data):
     data[row['home_team_name']] = {}
     # Iterando sobre o número de rodadas
     for i in range (39):
         data[row['home_team_name']][str(i)] = {}
         # Iterando sobre o número de colunas
         for metrica in df_times.columns:
            data[row['home_team_name']][str(i)][metrica] = 0

In [24]:
'''
consultar o dataset PARTIDAS

 atributos gerais
    'total_goal_count' -> contar gols do time
    'goals_scored_per_match' -> contar gols do time e dividir pelo número de jogos
    'goals_conceded_per_match' -> contar gols do time e dividir pelo número de jogos
    'over15_count' -> somar gols do mandante e gols do visitante e ver se é maior que 1
    'under15_count' -> somar gols do mandante e gols do visitante e ver se é menor que 1
    'clean_sheet_percentage' -> contar o número de partidas sem tomar gol e dividir
                                pelo número de partidas jogadas

 atributos casa
    'total_goal_count_home'
    'goals_scored_per_match_home'
    'goals_conceded_per_match_home'
    'over15_count_home'
    'under15_count_home'
    'clean_sheet_percentage_home'

 atributos fora
    'total_goal_count_away'
    'goals_conceded_per_match_away'
    'goals_scored_per_match_away'
    'over15_count_away'
    'under15_count_away'
    'clean_sheet_percentage_away'

Como os datasets pelos quais eu iterei são relativamente pequenos,
usei o método .iterrows() nessa e em outras iterações, apesar de
ele ser um método custoso
'''

# Primeiro 'for' para as rodadas
for index, row in partidas.iterrows():
  time_mandante = row['home_team_name']
  time_visitante = row['away_team_name']

  dic_mandante[time_mandante][0] += row['home_team_goal_count'] # gols feitos em casa pelo time da casa
  dic_mandante[time_mandante][1] += row['away_team_goal_count'] # gols sofridos em casa pelo time da casa
  dic_mandante[time_mandante][2] += 1  # número de partidas jogadas em casa pelo time
  dic_mandante[time_mandante][3] += 1 if row['away_team_goal_count'] == 0 else 0 # número de cleansheets em casa do time

  dic_visitante[time_visitante][0] += row['away_team_goal_count'] # gols feitos fora de casa pelo time de fora
  dic_visitante[time_visitante][1] += row['home_team_goal_count'] # gols sofridos fora de casa pelo time de fora
  dic_visitante[time_visitante][2] += 1  # número de partidas jogadas fora de casa pelo time
  dic_visitante[time_visitante][3] += 1 if row['home_team_goal_count'] == 0 else 0  # número de cleansheets fora de casa do time

  jogos_time_visitante = dic_mandante[time_visitante][2] + dic_visitante[time_visitante][2]
  jogos_time_mandante = dic_mandante[time_mandante][2] + dic_visitante[time_mandante][2]


  # Segundo 'for' para os atributos
  for metrica in df_times.columns:
      metrica_mandante = metrica.find('home')
      metrica_visitante = metrica.find('away')

      # Atributos do mandante
      if metrica_mandante != -1:
        data[time_visitante][str(jogos_time_visitante)][metrica] = data[time_visitante][str(jogos_time_visitante - 1)][metrica]
        data[time_mandante][str(jogos_time_mandante)][metrica] = data[time_mandante][str(jogos_time_mandante - 1)][metrica]
        if(metrica == 'total_goal_count_home'):
          data[time_mandante][str(jogos_time_mandante)][metrica] += row['home_team_goal_count'] + row['away_team_goal_count']

        elif(metrica == 'goals_scored_per_match_home'):
          data[time_mandante][str(jogos_time_mandante)][metrica] = (dic_mandante[time_mandante][0] / dic_mandante[time_mandante][2])

        elif(metrica == 'goals_conceded_per_match_home'):
          data[time_mandante][str(jogos_time_mandante)][metrica] = (dic_mandante[time_mandante][1] / dic_mandante[time_mandante][2])

        elif(metrica == 'under15_count_home'):
          data[time_mandante][str(jogos_time_mandante)][metrica] +=  (0 if (row['home_team_goal_count'] + row['away_team_goal_count']) > 1 else 1)

        elif(metrica == 'clean_sheet_percentage_home'):
          data[time_mandante][str(jogos_time_mandante)][metrica] =  (dic_mandante[time_mandante][3] * 100) / dic_mandante[time_mandante][2]

        else:
            data[time_mandante][str(jogos_time_mandante)][metrica] += (1 if (row['home_team_goal_count'] + row['away_team_goal_count']) > 1 else 0)

      # Atributos do visitante
      elif metrica_visitante != -1:
        data[time_visitante][str(jogos_time_visitante)][metrica] = data[time_visitante][str(jogos_time_visitante - 1)][metrica]
        data[time_mandante][str(jogos_time_mandante)][metrica] = data[time_mandante][str(jogos_time_mandante - 1)][metrica]
        if(metrica == 'total_goal_count_away'):
          data[time_visitante][str(jogos_time_visitante)][metrica] += row['home_team_goal_count'] + row['away_team_goal_count']

        elif(metrica == 'goals_scored_per_match_away'):
          data[time_visitante][str(jogos_time_visitante)][metrica] = (dic_visitante[time_visitante][0] / dic_visitante[time_visitante][2])

        elif(metrica == 'goals_conceded_per_match_away'):
          data[time_visitante][str(jogos_time_visitante)][metrica] = (dic_visitante[time_visitante][1] / dic_visitante[time_visitante][2])

        elif(metrica == 'under15_count_away'):
          data[time_visitante][str(jogos_time_visitante)][metrica] += (0 if (row['home_team_goal_count'] + row['away_team_goal_count']) > 1 else 1)

        elif(metrica == 'clean_sheet_percentage_away'):
          data[time_visitante][str(jogos_time_visitante)][metrica] = (dic_visitante[time_visitante][3] * 100) / dic_visitante[time_visitante][2]

        else:
            data[time_visitante][str(jogos_time_visitante)][metrica] += (1 if (row['home_team_goal_count'] + row['away_team_goal_count']) > 1 else 0)

      # Atributos gerais
      else:
        if(metrica == 'total_goal_count'):
          data[time_mandante][str(jogos_time_mandante)][metrica] = (row['home_team_goal_count'] + row['away_team_goal_count']) + data[time_mandante][str(jogos_time_mandante - 1)][metrica]
          data[time_visitante][str(jogos_time_visitante)][metrica] = (row['home_team_goal_count'] + row['away_team_goal_count']) + data[time_visitante][str(jogos_time_visitante - 1)][metrica]

        elif(metrica == 'goals_scored_per_match'):
          data[time_mandante][str(jogos_time_mandante)][metrica] = ((dic_mandante[time_mandante][0] + dic_visitante[time_mandante][0]) / jogos_time_mandante)
          data[time_visitante][str(jogos_time_visitante)][metrica] = ((dic_mandante[time_visitante][0] + dic_visitante[time_visitante][0]) / jogos_time_visitante)

        elif(metrica == 'goals_conceded_per_match'):
          data[time_mandante][str(jogos_time_mandante)][metrica] = ((dic_mandante[time_mandante][1] + dic_visitante[time_mandante][1]) / jogos_time_mandante)
          data[time_visitante][str(jogos_time_visitante)][metrica] = ((dic_mandante[time_visitante][1] + dic_visitante[time_visitante][1]) / jogos_time_visitante)

        elif(metrica == 'under15_count'):
          data[time_mandante][str(jogos_time_mandante)][metrica] = data[time_mandante][str(jogos_time_mandante - 1)][metrica] +  (0 if (row['home_team_goal_count'] + row['away_team_goal_count']) > 1 else 1)
          data[time_visitante][str(jogos_time_visitante)][metrica] = data[time_visitante][str(jogos_time_visitante - 1)][metrica] + (0 if (row['home_team_goal_count'] + row['away_team_goal_count']) > 1 else 1)

        elif(metrica == 'clean_sheet_percentage'):

          data[time_mandante][str(jogos_time_mandante)][metrica] = ((dic_mandante[time_mandante][3] + dic_visitante[time_mandante][3])  * 100) / (dic_mandante[time_mandante][2] + dic_visitante[time_mandante][2])
          data[time_visitante][str(jogos_time_visitante)][metrica] = ((dic_mandante[time_visitante][3] + dic_visitante[time_visitante][3])  * 100) / (dic_mandante[time_visitante][2] + dic_visitante[time_visitante][2])

        else:
          data[time_mandante][str(jogos_time_mandante)][metrica] = data[time_mandante][str(jogos_time_mandante - 1)][metrica] + (1 if (row['home_team_goal_count'] + row['away_team_goal_count']) > 1 else 0)
          data[time_visitante][str(jogos_time_visitante)][metrica] = data[time_visitante][str(jogos_time_visitante - 1)][metrica] + (1 if (row['home_team_goal_count'] + row['away_team_goal_count']) > 1 else 0)


In [25]:
'''
Transformando o dicionário 'data' em
uma matriz e posteriormente, convertendo-a
em um DataFrame
'''
data_2d = []
for time, jogos in data.items():
    for jogo, metricas in jogos.items():
        for metrica, valor in metricas.items():
            data_2d.append([time, jogo, metrica, valor])
df_data = pd.DataFrame(data_2d, columns=['Time', 'Número do jogo', 'Métrica', 'Valor'])

In [26]:
df_data['Valor'] = df_data['Valor'].round(2)

### Coletando dados relativos à forma usando webscraping


In [27]:
'''
Dicionário para guardar os últimos resultados
de cada time em cada rodada
'''
forma = {}
for index, row in partidas.iterrows():
  if(row['home_team_name'] not in forma):
     time = row['home_team_name']
     if(row['home_team_name'] in ['Liverpool', 'Everton', 'Chelsea']):
        time = row['home_team_name'] + ' FC'
     forma[time] = {}
     for i in range (38):
         forma[time][i + 1] = {'forma' : []}

In [28]:
'''
Dicionário para manter registro de quantos jogos
cada time já jogou
'''
rodadas =  {}
for index, row in partidas.iterrows():
  if(row['home_team_name'] not in rodadas):
     time = row['home_team_name']
     if(row['home_team_name'] in ['Liverpool', 'Everton', 'Chelsea']):
        time = row['home_team_name'] + ' FC'
     rodadas[time] = 1

In [29]:
url = 'https://footballdatabase.com/league-scores/england-premier-league-2018-19/6'

In [30]:
'''
Função que extrai os jogos de mais de uma página
de forma recursiva e preenche o dicionário 'forma',
associando a cada time e rodada, uma lista de tuplas, que
representam os últimos resultados daquele time(o primeiro elemento
da tupla representa o número de gols que a equipe marcou e o segundo é o
número de gols que a equipe sofreu)
'''
def extract(url, num_pagina):
  if num_pagina == 0:
    return

  pagina = r.get(url)
  conteudo = pagina.text
  data = bs(conteudo, 'html.parser')
  caixa = data.find_all('div', class_='col-md-8')[-1]
  confrontos = caixa.find_all('div', class_='club-gamelist-match')

  for i in range(len(confrontos) - 1, -1, -1):
    confronto = confrontos[i]
    score = confronto.find('div', class_='club-gamelist-match-score text-center')
    gol_mandante = int(score.text[0])
    gol_visitante = int(score.text[4])
    time_mandante = confronto.find_all('a')[0].text
    time_visitante = confronto.find_all('a')[1].text

    forma[time_mandante][rodadas[time_mandante]]['adv'] = time_visitante
    forma[time_mandante][rodadas[time_mandante]]['local'] = 'home'
    if(rodadas[time_mandante] < 38):
      forma[time_mandante][rodadas[time_mandante] + 1]['forma'].append((gol_mandante, gol_visitante))
      jogos_recentes = forma[time_mandante][rodadas[time_mandante]]['forma']
      for i in range(min(len(jogos_recentes), 4)):
        forma[time_mandante][rodadas[time_mandante] + 1]['forma'].append(jogos_recentes[i])
      rodadas[time_mandante] += 1

    forma[time_visitante][rodadas[time_visitante]]['adv'] = time_mandante
    forma[time_visitante][rodadas[time_visitante]]['local'] = 'away'
    if(rodadas[time_visitante] < 38):
      forma[time_visitante][rodadas[time_visitante] + 1]['forma'].append((gol_visitante, gol_mandante))
      jogos_recentes = forma[time_visitante][rodadas[time_visitante]]['forma']
      for i in range(min(len(jogos_recentes), 4)):
        forma[time_visitante][rodadas[time_visitante] + 1]['forma'].append(jogos_recentes[i])
      rodadas[time_visitante] += 1

  pagina_anterior = num_pagina - 1
  url2 = url.replace(f'/{num_pagina}', f'/{pagina_anterior}')
  extract(url2, pagina_anterior)

In [31]:
extract(url, 6)

In [32]:
'''
Transformando o dicionário 'forma' em
uma matriz e posteriormente, convertendo-a
para um DataFrame, onde os números entre parênteses
nas colunas com prefixo 'Gols marcados/sofridos' representam
o número de rodadas atrás em que aquele time sofreu ou marcou
determinada quantidade de gols
'''
forma_2d = []
for time, rodadas in forma.items():
  for rodada, info in rodadas.items():
      linha = [time,rodada]
      linha.append(info['adv'])
      linha.append(info['local'])
      for atributo, valor in info.items():
        if(atributo == 'forma'):
          for gols_marcados, gols_sofridos in valor:
            linha.append(gols_marcados)
            linha.append(gols_sofridos)
      forma_2d.append(linha)

df_forma = pd.DataFrame(forma_2d, columns=['Time', 'Rodada', 'Adversário', 'Local','goals_scored_home_team(1)', 'goals_suffered_home_team(1)',
                                         'goals_scored_home_team(2)',  'goals_suffered_home_team(2)', 'goals_scored_home_team(3)', 'goals_suffered_home_team(3)',
                                          'goals_scored_home_team(4)',  'goals_suffered_home_team(4)', 'goals_scored_home_team(5)', 'goals_suffered_home_team(5)',], dtype=object)

df_forma_mandantes = df_forma[df_forma['Local'] == 'home']
df_forma_visitantes = df_forma[df_forma['Local'] == 'away']
linhas = df_forma_mandantes.shape[0]
colunas = df_forma_mandantes.shape[1]
for i in range(10):
  if(i % 2 == 0):
    df_forma_mandantes.insert(colunas + i, f'goals_scored_away_team({i//2 + 1})', np.zeros(linhas), True)
  else:
    df_forma_mandantes.insert(colunas + i, f'goals_suffered_away_team({i//2 + 1})', np.zeros(linhas), True)

for i in range(linhas):
  linha = df_forma_mandantes.iloc[i : i + 1, 0 : 4]
  time = linha.iloc[0, 0]
  rodada = linha.iloc[0, 1]
  adv = linha.iloc[0, 2]
  local = linha.iloc[0, 3]

  local2 = 'home' if local == 'away' else 'away'
  cond = (df_forma_visitantes['Time'] == adv) & (df_forma_visitantes['Adversário'] == time) & (df_forma_visitantes['Local'] == local2)
  nova_linha = df_forma_visitantes[cond].iloc[:, 4 : 14]

  for j in range(0, 10, 2):
    gols_marcados = nova_linha.iloc[0, j]
    gols_sofridos = nova_linha.iloc[0, j + 1]
    df_forma_mandantes.iloc[i, j + 14] = gols_marcados
    df_forma_mandantes.iloc[i, j + 15] = gols_sofridos

df_forma_mandantes

,Time,Rodada,Adversário,Local,goals_scored_home_team(1),goals_suffered_home_team(1),goals_scored_home_team(2),goals_suffered_home_team(2),goals_scored_home_team(3),goals_suffered_home_team(3),...,goals_scored_away_team(1),goals_suffered_away_team(1),goals_scored_away_team(2),goals_suffered_away_team(2),goals_scored_away_team(3),goals_suffered_away_team(3),goals_scored_away_team(4),goals_suffered_away_team(4),goals_scored_away_team(5),goals_suffered_away_team(5)
0,Manchester United,1,Leicester City,home,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Manchester United,3,Tottenham Hotspur,home,2,3,2,1,None,None,...,3.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,Manchester United,6,Wolverhampton Wanderers,home,2,1,2,0,0,3,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,2.0,2.0
7,Manchester United,8,Newcastle United,home,1,3,1,1,2,1,...,0.0,2.0,0.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0
9,Manchester United,10,Everton FC,home,2,2,3,2,1,3,...,2.0,0.0,2.0,1.0,3.0,0.0,0.0,2.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,Crystal Palace,30,Brighton & Hove Albion,home,3,1,1,3,4,1,...,1.0,0.0,1.0,2.0,1.0,3.0,0.0,0.0,2.0,4.0
752,Crystal Palace,31,Huddersfield Town,home,1,2,3,1,1,3,...,3.0,4.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,2.0
755,Crystal Palace,34,Manchester City,home,1,0,0,2,2,0,...,2.0,0.0,2.0,0.0,3.0,1.0,1.0,0.0,1.0,0.0
757,Crystal Palace,36,Everton FC,home,3,2,1,3,1,0,...,4.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0,2.0,0.0


In [33]:
'''
 Limpando os dados e deixando
 os valores do time em conformidade
 como o dataset partidas
'''

for column in range(14, 24): # colunas que dizem respeito aos gols marcados e sofridos pelo time visitante
    df_forma_mandantes.iloc[:, column] = [int(val) if pd.notnull(val) else 'Não houve jogo' for val in df_forma_mandantes.iloc[:, column]]
df_forma_mandantes = df_forma_mandantes.fillna('Não houve jogo')
df_forma_mandantes['Time'].replace({'Everton FC' : 'Everton', 'Liverpool FC' : 'Liverpool', 'Chelsea FC' : 'Chelsea'}, inplace = True)
df_forma_mandantes['Adversário'].replace({'Everton FC' : 'Everton', 'Liverpool FC' : 'Liverpool', 'Chelsea FC' : 'Chelsea'}, inplace = True)
df_forma_mandantes.reset_index(drop = True, inplace = True)

In [34]:
df_forma_mandantes = df_forma_mandantes.sort_values(by = ['Rodada'])

In [35]:
'''
 Criação de um novo dataframe a partir das
 informações do df_forma_mandantes, porém
 com as linhas seguindo a mesma ordem dos
 jogos no dataframe partidas para facilitar
 a junção dos dois dataframes
'''
df_forma_mandantes_final = pd.DataFrame()
linhas = df.shape[0]
for i in range(linhas):
  time_mandante_i = df.iloc[i, 1]
  time_visitante_i = df.iloc[i, 2]
  linha  = df_forma_mandantes[(df_forma_mandantes['Time'] == time_mandante_i) & (df_forma_mandantes['Adversário'] == time_visitante_i)].iloc[:, 4:] # colunas que contém a forma dos times
  df_forma_mandantes_final = pd.concat([df_forma_mandantes_final, linha], axis = 0)
df_forma_mandantes_final.reset_index(drop = True, inplace = True)
df_forma_mandantes_final

,goals_scored_home_team(1),goals_suffered_home_team(1),goals_scored_home_team(2),goals_suffered_home_team(2),goals_scored_home_team(3),goals_suffered_home_team(3),goals_scored_home_team(4),goals_suffered_home_team(4),goals_scored_home_team(5),goals_suffered_home_team(5),goals_scored_away_team(1),goals_suffered_away_team(1),goals_scored_away_team(2),goals_suffered_away_team(2),goals_scored_away_team(3),goals_suffered_away_team(3),goals_scored_away_team(4),goals_suffered_away_team(4),goals_scored_away_team(5),goals_suffered_away_team(5)
0,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo
1,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo
2,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo
3,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo
4,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,3,2,5,0,2,0,2,0,3,1,1,0,2,1,3,1,0,0,1,3
376,1,1,1,1,0,2,0,4,2,1,2,3,0,1,0,2,2,0,0,2
377,0,3,3,3,1,1,1,3,3,1,1,1,0,5,1,2,0,4,1,4
378,0,1,0,1,1,0,0,1,4,0,2,0,0,0,4,0,0,2,1,0


In [36]:
df

,timestamp,home_team_name,away_team_name,referee,Home Team Pre-Match xG,Away Team Pre-Match xG,over_15_percentage_pre_match,odds_ft_over15,average_goals_per_match_pre_match,goals
0,1533927600,Manchester United,Leicester City,Andre Marriner,0.00,0.00,0,1.33,0.00,3
1,1533987000,Newcastle United,Tottenham Hotspur,Martin Atkinson,0.00,0.00,0,1.31,0.00,3
2,1533996000,AFC Bournemouth,Cardiff City,Kevin Friend,0.00,0.00,0,1.31,0.00,2
3,1533996000,Fulham,Crystal Palace,Mike Dean,0.00,0.00,0,1.28,0.00,2
4,1533996000,Huddersfield Town,Chelsea,Chris Kavanagh,0.00,0.00,0,1.29,0.00,3
...,...,...,...,...,...,...,...,...,...,...
375,1557669600,Liverpool,Wolverhampton Wanderers,Martin Atkinson,1.96,1.27,81,1.18,2.92,2
376,1557669600,Manchester United,Cardiff City,Jonathan Moss,1.67,1.10,78,1.12,2.72,2
377,1557669600,Southampton,Huddersfield Town,Lee Probert,1.46,1.11,84,1.19,3.06,2
378,1557669600,Tottenham Hotspur,Everton,Andre Marriner,1.71,1.32,64,1.22,2.53,4


### Junção de todos os dados em um único dataframe

In [37]:
# Junção das colunas escolhidas do dataset Partidas com os dados coletados com webscraping
df = pd.concat([df, df_forma_mandantes_final], axis = 1)
df

,timestamp,home_team_name,away_team_name,referee,Home Team Pre-Match xG,Away Team Pre-Match xG,over_15_percentage_pre_match,odds_ft_over15,average_goals_per_match_pre_match,goals,...,goals_scored_away_team(1),goals_suffered_away_team(1),goals_scored_away_team(2),goals_suffered_away_team(2),goals_scored_away_team(3),goals_suffered_away_team(3),goals_scored_away_team(4),goals_suffered_away_team(4),goals_scored_away_team(5),goals_suffered_away_team(5)
0,1533927600,Manchester United,Leicester City,Andre Marriner,0.00,0.00,0,1.33,0.00,3,...,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo
1,1533987000,Newcastle United,Tottenham Hotspur,Martin Atkinson,0.00,0.00,0,1.31,0.00,3,...,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo
2,1533996000,AFC Bournemouth,Cardiff City,Kevin Friend,0.00,0.00,0,1.31,0.00,2,...,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo
3,1533996000,Fulham,Crystal Palace,Mike Dean,0.00,0.00,0,1.28,0.00,2,...,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo
4,1533996000,Huddersfield Town,Chelsea,Chris Kavanagh,0.00,0.00,0,1.29,0.00,3,...,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo,Não houve jogo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1557669600,Liverpool,Wolverhampton Wanderers,Martin Atkinson,1.96,1.27,81,1.18,2.92,2,...,1,0,2,1,3,1,0,0,1,3
376,1557669600,Manchester United,Cardiff City,Jonathan Moss,1.67,1.10,78,1.12,2.72,2,...,2,3,0,1,0,2,2,0,0,2
377,1557669600,Southampton,Huddersfield Town,Lee Probert,1.46,1.11,84,1.19,3.06,2,...,1,1,0,5,1,2,0,4,1,4
378,1557669600,Tottenham Hotspur,Everton,Andre Marriner,1.71,1.32,64,1.22,2.53,4,...,2,0,0,0,4,0,0,2,1,0


In [39]:
# Reset do dicionário que registra o número de jogos de cada time
for x in lista_times:
  jogos[x] = 0

# Juntando o dataframe resultante com o os dados inferidos através do dataset Partidas
for index, row in df.iterrows():
  time_mandante = row['home_team_name']
  time_visitante = row['away_team_name']
  df_data_mandante = df_data[(df_data['Número do jogo'] == str(jogos[time_mandante])) & (df_data['Time'] == time_mandante)]
  df_data_visitante = df_data[(df_data['Número do jogo'] == str(jogos[time_visitante])) & (df_data['Time'] == time_visitante)]

  for metrica in df_data_mandante['Métrica']:
    metrica_visitante = metrica.find('away')
    if(metrica_visitante == -1):
        df_data_mandante_aux = df_data_mandante[df_data_mandante['Métrica'] == metrica].Valor
        value = np.array(df_data_mandante_aux)[0]
        df.loc[index : index, (metrica + '_homeTeam')] = value

  for metrica in df_data_visitante['Métrica']:
    metrica_mandante = metrica.find('home')
    if(metrica_mandante == -1):
        df_data_visitante_aux = df_data_visitante[df_data_visitante['Métrica'] == metrica].Valor
        value = np.array(df_data_visitante_aux)[0]
        df.loc[index : index, (metrica + '_awayTeam')] = value

  jogos[time_mandante] += 1
  jogos[time_visitante] += 1

In [41]:
df.columns

Index(['timestamp', 'home_team_name', 'away_team_name', 'referee',
       'Home Team Pre-Match xG', 'Away Team Pre-Match xG',
       'over_15_percentage_pre_match', 'odds_ft_over15',
       'average_goals_per_match_pre_match', 'goals',
       'goals_scored_home_team(1)', 'goals_suffered_home_team(1)',
       'goals_scored_home_team(2)', 'goals_suffered_home_team(2)',
       'goals_scored_home_team(3)', 'goals_suffered_home_team(3)',
       'goals_scored_home_team(4)', 'goals_suffered_home_team(4)',
       'goals_scored_home_team(5)', 'goals_suffered_home_team(5)',
       'goals_scored_away_team(1)', 'goals_suffered_away_team(1)',
       'goals_scored_away_team(2)', 'goals_suffered_away_team(2)',
       'goals_scored_away_team(3)', 'goals_suffered_away_team(3)',
       'goals_scored_away_team(4)', 'goals_suffered_away_team(4)',
       'goals_scored_away_team(5)', 'goals_suffered_away_team(5)',
       'total_goal_count_homeTeam', 'total_goal_count_home_homeTeam',
       'goals_scored_per

In [40]:
df

,timestamp,home_team_name,away_team_name,referee,Home Team Pre-Match xG,Away Team Pre-Match xG,over_15_percentage_pre_match,odds_ft_over15,average_goals_per_match_pre_match,goals,...,goals_scored_per_match_awayTeam,goals_scored_per_match_away_awayTeam,goals_conceded_per_match_awayTeam,goals_conceded_per_match_away_awayTeam,over15_count_awayTeam,over15_count_away_awayTeam,under15_count_awayTeam,under15_count_away_awayTeam,clean_sheet_percentage_awayTeam,clean_sheet_percentage_away_awayTeam
0,1533927600,Manchester United,Leicester City,Andre Marriner,0.00,0.00,0,1.33,0.00,3,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1533987000,Newcastle United,Tottenham Hotspur,Martin Atkinson,0.00,0.00,0,1.31,0.00,3,...,0.00,0.0,4.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1533996000,AFC Bournemouth,Cardiff City,Kevin Friend,0.00,0.00,0,1.31,0.00,2,...,0.50,0.0,3.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0
3,1533996000,Fulham,Crystal Palace,Mike Dean,0.00,0.00,0,1.28,0.00,2,...,0.67,0.5,3.0,3.5,3.0,2.0,0.0,0.0,0.0,0.0
4,1533996000,Huddersfield Town,Chelsea,Chris Kavanagh,0.00,0.00,0,1.29,0.00,3,...,0.50,0.5,2.5,3.5,3.0,2.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1557669600,Liverpool,Wolverhampton Wanderers,Martin Atkinson,1.96,1.27,81,1.18,2.92,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,1557669600,Manchester United,Cardiff City,Jonathan Moss,1.67,1.10,78,1.12,2.72,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,1557669600,Southampton,Huddersfield Town,Lee Probert,1.46,1.11,84,1.19,3.06,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,1557669600,Tottenham Hotspur,Everton,Andre Marriner,1.71,1.32,64,1.22,2.53,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Carregamento em um arquivo .csv

In [39]:
csv_path = "/content/teams_stats_for_goals_complete.csv"
df.to_csv(csv_path)